### Algorithm:

1. Search the field replacing keywords in the URL

2. Get all the names and URLs from the page

3. Check if the keyword appears in each URL name

4. If the keyword appears in the URL name, add the name and URL to the list

5. Get 'next' URL and repeat the process until the keyword does not appear in the URL name

6. Get all the pages of the urls collected and get the
date filed for each URL

7. Sort the list by date filed

8. Remove the entries more than 30 days old

9. Print the list of names and URLs


In [115]:
import pandas as pd
import bs4
import requests

In [120]:
names_and_links = []
keyword = "circuit"
header_link = 'https://search.sunbiz.org/'
r = requests.get(f'{header_link}Inquiry/CorporationSearch/SearchResults/EntityName/{keyword}/Page1?searchNameOrder={keyword.upper()}')
r.raise_for_status()

soup = bs4.BeautifulSoup(r.text, 'html.parser')


print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te

In [136]:
search_results = soup.find_all(attrs = {'id': 'search-results'})
nali = search_results[0].find_all('tr')
nali = [[nali[i].td.text, nali[i].a['href']] for i in range(1, len(nali))]

print(nali, len(nali), sep = '\n')

[['CIRCUIT CLERK TECH SOLUTIONS LLC', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=ForwardList&searchNameOrder=CIRCUITCLERKTECHSOLUTIONS%20L230001103160&aggregateId=flal-l23000110316-54912995-3e71-4d04-8e82-667d4e44cad6&searchTerm=circuit&listNameOrder=CIRCUITCLERKTECHSOLUTIONS%20L230001103160'], ['CIRCUIT CLINICAL SOLUTIONS INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=ForwardList&searchNameOrder=CIRCUITCLINICALSOLUTIONS%20F230000069200&aggregateId=forp-f23000006920-ab1eba4b-0f30-4d2c-9f0b-497a39edd62f&searchTerm=circuit&listNameOrder=CIRCUITCLERKTECHSOLUTIONS%20L230001103160'], ['CIRCUIT CLINICAL SOLUTIONS INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=ForwardList&searchNameOrder=CIRCUITCLINICALSOLUTIONS%20W230001306100&aggregateId=reject-w23000130610-49b31fe4-75a1-4bd3-8b35-bca6ddca2860&searchTerm=circuit&listNameOrder=CIRCUITCLERKTECHSOLUTIONS%20L230001103

In [137]:
def check_name(keyword, namelink_list):
    length = len(namelink_list)
    status = []
    for idx in range(length):
        if keyword.lower() in namelink_list[idx][0].lower():
            status.append((namelink_list[idx][0], True))
        else:
            status.append((namelink_list[idx][0], False))
    return status
checked_list = check_name(keyword, nali)

In [138]:
checked_dt = pd.DataFrame(checked_list, columns = ['Name', 'Match'])
print(checked_dt)

                                                 Name  Match
0                    CIRCUIT CLERK TECH SOLUTIONS LLC   True
1                     CIRCUIT CLINICAL SOLUTIONS INC.   True
2                     CIRCUIT CLINICAL SOLUTIONS INC.   True
3                               THE-CIRCUIT.COM, INC.   True
4                              CIRCUIT CONCEPTS, INC.   True
5                                  CIRCUIT CONNEX LLC   True
6                    CIRCUIT CONSTRUCTION CORPORATION   True
7                               CIRCUIT COUNTRY CORP.   True
8                           CIRCUIT CRAFT CORPORATION   True
9                                   CIRCUIT DANCE LLC   True
10                                  CIRCUIT DEALS LLC   True
11                                CIRCUIT DEPOT, INC.   True
12                     CIRCUIT DEPOT FREE ZONE NV LLC   True
13                        CIRCUIT DESIGN INCORPORATED   True
14                              CIRCUIT DESIGNS, INC.   True
15                      

In [139]:
t_num = checked_dt[checked_dt['Match'] == True].count()
f_num = checked_dt[checked_dt['Match'] == False].count()
t_num[0]/(t_num[0] + f_num[0])

/tmp/ipykernel_13246/1113156225.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  t_num[0]/(t_num[0] + f_num[0])


1.0

In [140]:
if (t_num[0]/(t_num[0] + f_num[0])) > 0.8:
    names_and_links.extend(nali)
    print('Next Page')
else:
    names_and_links.extend(nali)
    # break statement
    print('Last Page')

print(names_and_links, len(names_and_links), sep = '\n')

Next Page


/tmp/ipykernel_13246/2932176759.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (t_num[0]/(t_num[0] + f_num[0])) > 0.8:


In [142]:
next_button = soup.find_all('div', attrs = {'class': 'navigationBarPaging'})
next_button_link = next_button[0].find_all('a')[-1]['href'].lstrip('/')
next_button_link

'Inquiry/CorporationSearch/SearchResults?InquiryType=EntityName&inquiryDirectionType=ForwardList&searchNameOrder=CIRCUITDUSOLEIL%20L110001270080&SearchTerm=circuit&entityId=L11000127008&listNameOrder=CIRCUITCLERKTECHSOLUTIONS%20L230001103160'

In [135]:
r = requests.get(header_link + next_button_link)
r.raise_for_status()

soup = bs4.BeautifulSoup(r.text, 'html.parser')


print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te

In [102]:
r = requests.get(header_link + names_and_links[0][1])
r.raise_for_status()

comps = bs4.BeautifulSoup(r.text, 'html.parser')


print(comps.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te

In [114]:
date_filed = comps.find_all('label', attrs = {'for': 'Detail_FileDate'})
date_filed = date_filed[0].next_sibling.text
date_filed

'07/22/1999'